In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [15]:
#read metadata
metadata = pd.read_csv("/Users/minjacvetkovski/Documents/DH MA/Collecting Data/A3/metadata.csv")

#new list for scraped sections
scraped_texts = []
chapter_list = []
page_num_list = []

#scrape all urls
for url in metadata['Url']:
    response = requests.get(url)
    document = BeautifulSoup(response.text, "html.parser")

    #chapter titles
    chap = document.find("h2", class_="readerhead2")
    if chap:
        raw_chap = chap.get_text(" ", strip=True)         
        cleaned_chap = re.sub(r'\s+', ' ', raw_chap).strip()
        chapter_list.append(cleaned_chap)
    else:
        chapter_list.append(None)

    #page numbers
    page = document.find("div", class_="pagefoot")
    if page:
        m = re.search(r"page\s+(\d+)", page.get_text())
        if m:
            page_num_list.append(int(m.group(1)))
        else:
            page_num_list.append(None)
    else:
        page_num_list.append(None)
        
    #text
    paragraphs = document.find_all("p", class_="reading")
    full_text = " ".join([p.get_text(" ", strip=True) for p in paragraphs])
    cleaned_text = " ".join(full_text.split())
    scraped_texts.append(cleaned_text)

#add scraped sections as new columns
metadata['Content'] = scraped_texts
metadata['Chapter'] = chapter_list
metadata['Page_num'] = page_num_list

#add word count (tokens)
metadata['Word_count'] = metadata['Content'].apply(lambda x: len(x.split()))

#view new table
metadata

,Author,Title,Year,Url,Content,Chapter,Page_num,Word_count
0,Herman Melville,Moby Dick,1851,https://www.literaturepage.com/read/mobydick-1...,"But look! here come more crowds, pacing straig...",CHAPTER 1: Loomings. (continued),13,575
1,Bram Stoker,Dracula,1897,https://www.literaturepage.com/read/dracula-32...,"""We Szekelys have a right to be proud, for in ...",CHAPTER 3 (continued),32,569
2,Oscar Wilde,The Picture of Dorian Gray,1890,https://www.literaturepage.com/read/doriangray...,She was extremely annoyed at the tone he had a...,CHAPTER 5 (continued),68,559
3,Charles Dickens,Oliver Twist,1837,https://www.literaturepage.com/read/olivertwis...,Oliver knew not the cause of this sudden excla...,Chapter 12 : IN WHICH OLIVER IS TAKEN BETTER C...,94,517
4,Thomas Hardy,The Woodlanders,1887,https://www.literaturepage.com/read/the-woodla...,The spot seemed now to be quite deserted. The ...,13. CHAPTER XIII. (continued),104,460
5,Henry James,The Portrait of a Lady,1880,https://www.literaturepage.com/read/portrait-o...,"He had been a very small boy when his father, ...",VOLUME I 5. CHAPTER V (continued),52,677
6,Mary Wollstonecraft Shelley,Frankenstein,1818,https://www.literaturepage.com/read/frankenste...,"From Italy they visited Germany and France. I,...",Chapter 1 (continued),20,785
7,Charlotte Bronte,Jane Eyre,1847,https://www.literaturepage.com/read/janeeyre-1...,"The promise of a smooth career, which my first...",12. CHAPTER XII,126,474
8,Louisa May Alcott,Little Women,1868,https://www.literaturepage.com/read/littlewome...,"As spring came on, a new set of amusements bec...",PART 1 10. CHAPTER TEN,113,493
9,Anne Bronte,The Tenant of Wildfowl Hall,1848,https://www.literaturepage.com/read/tenant-of-...,Miss Wilson drew herself up with a look of fre...,9. CHAPTER IX (continued),76,521


In [16]:
#save new corpus
metadata.to_csv("/Users/minjacvetkovski/Documents/DH MA/Collecting Data/A3/corpus.csv", index=False)